# LDA - Linear Discriminent Analysis

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import warnings
warnings.filterwarnings("ignore")

In [5]:
dataset1=pd.read_csv("CKD.csv",index_col=None)
df=dataset1
df = pd.get_dummies(df, drop_first=True)
df

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,False,False,False,False,False,False,True,True,False,True
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,True,False,False,False,False,False,True,False,False,True
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,True,False,False,False,False,False,True,False,False,True
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,True,False,False,False,False,False,True,False,True,True
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,True,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,True,False,False,False,False,False,True,False,False,True
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,True,False,False,True,True,False,True,False,True,True
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,True,False,False,True,True,False,False,False,False,True
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,True,False,False,True,True,False,True,False,True,True


In [7]:
indep_x = df.drop('classification_yes',axis = 1)
dep_y = df['classification_yes']

In [9]:
def split_scaler(indep_x,dep_y):
    x_train,x_test,y_train,y_test = train_test_split(indep_x,dep_y,test_size = 0.2, random_state=0)
    sc=StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    return x_train,x_test,y_train,y_test

def cm_prediction(classifier,x_test,y_test):
    y_pred = classifier.predict(x_test)
    cm = confusion_matrix(y_test,y_pred)
    Accuracy = accuracy_score(y_test,y_pred)
    Report = classification_report(y_test,y_pred)
    return cm,Accuracy,Report

def logistic(x_train,y_train,x_test,y_test):
    classifier = LogisticRegression(random_state=0)
    classifier.fit(x_train,y_train)
    return cm_prediction(classifier,x_test,y_test)

def svm_linear(x_train,y_train,x_test,y_test):
    classifier = SVC(kernel='linear',random_state=0)
    classifier.fit(x_train,y_train)
    return cm_prediction(classifier,x_test,y_test)

def svm_nonlinear(x_train,y_train,x_test,y_test):
    classifier = SVC(kernel = 'rbf', random_state=0)
    classifier.fit(x_train,y_train)
    return cm_prediction(classifier,x_test,y_test)

def knn(x_train,y_train,x_test,y_test):
    classifier = KNeighborsClassifier(n_neighbors=5, metric = 'minkowski')
    classifier.fit(x_train,y_train)
    return cm_prediction(classifier,x_test,y_test)

def naive(x_train,y_train,x_test,y_test):   
    classifier = GaussianNB()
    classifier.fit(x_train,y_train)
    return cm_prediction(classifier,x_test,y_test)

def decision(x_train,y_train,x_test,y_test):
    classifier = DecisionTreeClassifier(criterion = 'entropy', random_state=0)
    classifier.fit(x_train,y_train)
    return cm_prediction(classifier,x_test,y_test)

def random(x_train,y_train,x_test,y_test):
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=0)
    classifier.fit(x_train,y_train)
    return cm_prediction(classifier,x_test,y_test)

def LDA_results(acclog,accsvml,accsvmnl,accknn,accnav,accdc,accrf):
    LDAdataframe = pd.DataFrame(index = ['Logistic', 'SVM', 'Decision', 'Random'], columns = ['Logistic', 'SVM_l', 'SVM_nl', 'KNN',
                                                                                              'Naive','Decision','Random'])
    for index in LDAdataframe.index:
        LDAdataframe['Logistic'][index] = acclog
        LDAdataframe['SVM_l'][index] = accsvml
        LDAdataframe['SVM_nl'][index] = accsvmnl
        LDAdataframe['KNN'][index] = accknn
        LDAdataframe['Naive'][index] = accnav
        LDAdataframe['Decision'][index] = accdc
        LDAdataframe['Random'][index] = accrf
    return LDAdataframe

In [11]:
acclog =[] 
accsvml = []
accsvmnl = []
accknn = []
accnav = [] 
accdc = []
accrf = []

In [13]:
# Train Test Split

x_train,x_test,y_train,y_test = split_scaler(indep_x,dep_y)

In [21]:
# Applying LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 1)
x_train_lda = lda.fit_transform(x_train,y_train)
x_test_lda = lda.transform(x_test)

In [23]:
x_train.shape[1]

27

In [29]:
cm,Accuracy,Report = logistic(x_train_lda, y_train, x_test_lda, y_test)
acclog.append(Accuracy)

cm,Accuracy,Report = svm_linear(x_train_lda, y_train, x_test_lda, y_test)
accsvml.append(Accuracy)

cm,Accuracy,Report = svm_nonlinear(x_train_lda, y_train, x_test_lda, y_test)
accsvmnl.append(Accuracy)

cm,Accuracy,Report = knn(x_train_lda, y_train, x_test_lda, y_test)
accknn.append(Accuracy)

cm,Accuracy,Report = naive(x_train_lda, y_train, x_test_lda, y_test)
accnav.append(Accuracy)

cm,Accuracy,Report = decision(x_train_lda, y_train, x_test_lda, y_test)
accdc.append(Accuracy)

cm,Accuracy,Report = random(x_train_lda, y_train, x_test_lda, y_test)
accrf.append(Accuracy)

result = LDA_results(acclog[0],accsvml[0],accsvmnl[0],accknn[0],accnav[0],accdc[0],accrf[0])

In [31]:
result

,Logistic,SVM_l,SVM_nl,KNN,Naive,Decision,Random
Logistic,0.975,0.9625,0.9625,0.95,0.975,0.9625,0.9625
SVM,0.975,0.9625,0.9625,0.95,0.975,0.9625,0.9625
Decision,0.975,0.9625,0.9625,0.95,0.975,0.9625,0.9625
Random,0.975,0.9625,0.9625,0.95,0.975,0.9625,0.9625


As the Accuracy is good in logistic we are going to create a model for logisitic regression